 # Criação de Fato e Dimensões a partir dos dados Silver

 Neste notebook, vamos:
 1. Ler a tabela **silver_health.Claims** (já em formato Delta).
 2. Criar a tabela fato **Fato_Claims** selecionando as colunas relevantes.
 3. Criar a dimensão de tempo (**Dim_Tempo**) a partir da data de início do período faturável.
 4. Criar a dimensão de seguradora (**Dim_Insurance**) a partir dos valores distintos do campo *insurance*.

 Os resultados serão salvos no schema **dwh** (Data Warehouse). Se necessário, crie o schema executando:


 ## 1. Criação da Tabela Fato: Fato_Claims

 Selecionaremos as seguintes colunas da tabela **silver_health.Claims**:
 - **id**: Identificador da claim.
 - **billablePeriod_start** e **billablePeriod_end**: Período faturável.
 - **created** e **update_date**: Datas de criação e atualização.
 - **total**: Valor total da claim (medida financeira).
 - **insurance**: Informação da seguradora.
 - **patient_reference**, **provider_display**, **provider_reference**, **priority_coding**, **type** e **use**: Outras informações relevantes.

In [8]:
gold_schema = "gold_health"

StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 10, Finished, Available, Finished)

 # Extração e Transformação das Colunas (total, type, insurance, priority_coding)

 Neste notebook, vamos:
 1. Ler a tabela `silver_health.Claims`.
 2. Converter as colunas `total` e `type`, que estão em formato string JSON, para estruturas (struct/array<struct>).
 3. Extrair subcampos de:
    - `total` (value, currency)
    - `type` (primeiro elemento do array, primeiro coding)
    - `insurance` (primeiro elemento do array, campos coverage, focal, sequence)
    - `priority_coding` (primeiro elemento do array, code e system)
 4. Selecionar colunas relevantes e exibir o resultado final.

In [9]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import (
    StructType, StructField, DoubleType, StringType, ArrayType
)

# 1) Definição de schemas para as colunas em formato JSON

# "total" é um JSON do tipo: {"value": 12733.22, "currency": "USD"}
total_schema = StructType([
    StructField("value", DoubleType(), True),
    StructField("currency", StringType(), True)
])

# "type" é um JSON do tipo: 
# [
#   {
#     "coding": [
#       {
#         "system": "http://terminology.hl7.org/CodeSystem/claim-type",
#         "code": "institutional"
#       }
#     ]
#   }
# ]
type_schema = ArrayType(StructType([
    StructField("coding", ArrayType(StructType([
        StructField("system", StringType(), True),
        StructField("code", StringType(), True)
    ])), True)
]))



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 11, Finished, Available, Finished)

 ## Leitura da Tabela silver_health.Claims

 Carregamos os dados em um DataFrame `claims_df`.


In [10]:

# %% [code]
claims_df = spark.table("silver_health.Claims")

# Visualização rápida do schema
claims_df.printSchema()



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 12, Finished, Available, Finished)

root
 |-- abatementDateTime: string (nullable = true)
 |-- activity: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- detail: struct (nullable = true)
 |    |    |    |-- code: struct (nullable = true)
 |    |    |    |    |-- coding: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- code: string (nullable = true)
 |    |    |    |    |    |    |-- display: string (nullable = true)
 |    |    |    |    |    |    |-- system: string (nullable = true)
 |    |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- location: struct (nullable = true)
 |    |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- reasonReference: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- reference: string (nullable = true)
 |    |    |    |-- status: string (nullable = true)
 |-- address: array

 ## 2. Conversão das Colunas "total" e "type" (String JSON) em Estruturas



In [11]:
claims_df = claims_df \
    .withColumn("total_parsed", from_json(col("total"), total_schema)) \
    .withColumn("type_parsed", from_json(col("type"), type_schema))



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 13, Finished, Available, Finished)


 Agora, `total_parsed` é um struct com `value` e `currency`, e `type_parsed` é um array de structs, cada um contendo um array `coding`.


 ## 3. Extração de Subcampos de insurance e priority_coding

 - **insurance**: Já é `array<struct<coverage:..., focal:..., sequence:...>>`
 - **priority_coding**: Geralmente `array<struct<code:..., system:...>>`

 Selecionamos o primeiro elemento do array (índice zero) em cada caso, assumindo que só há um registro de interesse por linha. Se existirem vários, considere usar `explode()` ou outro método.



In [12]:
fato_claims_df = claims_df.select(
    col("id"),
    col("billablePeriod_start"),
    col("billablePeriod_end"),
    col("created"),
    col("update_date"),
    
    # total (já convertido para struct)
    col("total_parsed.value").alias("total_value"),
    col("total_parsed.currency").alias("total_currency"),
    
    # type (já convertido para array<struct<coding: array<struct<system, code>>>>)
    # Pegamos o 1º elemento do array e o 1º coding
    col("type_parsed")[0].coding[0].system.alias("type_system"),
    col("type_parsed")[0].coding[0].code.alias("type_code"),
    
    # insurance (array<struct<coverage: struct<display, reference>, focal, sequence>>)
    col("insurance")[0].coverage.display.alias("coverage_display"),
    col("insurance")[0].coverage.reference.alias("coverage_reference"),
    col("insurance")[0].focal.alias("coverage_focal"),
    col("insurance")[0].sequence.alias("coverage_sequence"),
    
    # priority_coding (array<struct<code, system>>)
    col("priority_coding")[0].code.alias("priority_code"),
    col("priority_coding")[0].system.alias("priority_system"),
    
    # Demais colunas desejadas
    col("patient_reference"),
    col("provider_display"),
    col("provider_reference"),
    col("use")
)



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 14, Finished, Available, Finished)

 ## 4. Exibir o Resultado Final




Salva a tabela fato como Delta no schema **GOLD**.



In [13]:
display(fato_claims_df.limit(10))

StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 32127f82-c958-4ccb-80db-ac2ad370c30c)

In [14]:
fato_claims_df.write.format("delta").mode("overwrite").saveAsTable(f"{gold_schema}.Fato_Claims")
print(f"Tabela fato '{gold_schema}.Fato_Claims' criada com sucesso!")


StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 16, Finished, Available, Finished)

Tabela fato 'gold_health.Fato_Claims' criada com sucesso!



 ## 2. Criação da Dimensão: Dim_Tempo

 Utilizaremos a coluna **billablePeriod_start** para criar a dimensão de tempo.
 Para cada data, extraímos:
 - Data completa
 - Ano
 - Mês
 - Trimestre
 - Dia do mês
 - Dia da semana



In [15]:
from pyspark.sql.functions import year, month, dayofmonth, quarter, dayofweek

dim_tempo_df = fato_claims_df.select("billablePeriod_start").distinct() \
    .withColumnRenamed("billablePeriod_start", "data") \
    .withColumn("ano", year("data")) \
    .withColumn("mes", month("data")) \
    .withColumn("trimestre", quarter("data")) \
    .withColumn("dia", dayofmonth("data")) \
    .withColumn("dia_semana", dayofweek("data"))

display(dim_tempo_df.limit(10))



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 54517097-08bd-4882-ad57-e9bbed833c56)

Salva a dimensão de tempo no schema **dwh**.

In [16]:

dim_tempo_df.write.format("delta").mode("overwrite").saveAsTable(f"{gold_schema}.Dim_Tempo")
print(f"Tabela dimensão '{gold_schema}.Dim_Tempo' criada com sucesso!")



StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 18, Finished, Available, Finished)

Tabela dimensão 'gold_health.Dim_Tempo' criada com sucesso!



 ## 3. Criação da Dimensão: Dim_Insurance

 A dimensão de seguradora será criada a partir dos valores distintos do campo **insurance**.
 Vamos gerar um identificador único para cada seguradora.

 **Observação:** Se o campo *insurance* já for um identificador, você pode usá-lo diretamente.



In [17]:
from pyspark.sql.functions import monotonically_increasing_id

# Cria a dimensão de seguradora a partir dos campos extraídos de insurance
# Usaremos "coverage_display" como nome do seguro
dim_insurance_df = fato_claims_df.select(
    "coverage_display", "coverage_reference", "coverage_focal", "coverage_sequence"
).distinct() \
  .withColumnRenamed("coverage_display", "nome_seguro") \
  .withColumn("id_seguro", monotonically_increasing_id())

display(dim_insurance_df.limit(10))

StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 838ba83a-a2c0-4c7e-995b-9a599db06a96)

Salva a dimensão de seguradora no schema **GOLD**.



In [18]:
dim_insurance_df.write.format("delta").mode("overwrite").saveAsTable(f"{gold_schema}.Dim_Insurance")
print(f"Tabela dimensão '{gold_schema}.Dim_Insurance' criada com sucesso!")


StatementMeta(, a1781d7e-e6ed-47cf-b990-884705d6eab4, 20, Finished, Available, Finished)

Tabela dimensão 'gold_health.Dim_Insurance' criada com sucesso!
